In [10]:
#pip install category_encoders

In [11]:
#pip install gradio

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
#from matplotlib import artist
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import roc_curve

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Chargement des données

In [3]:
data = pd.read_csv('/content/data3 (2).csv')
data = data.drop(columns = 'hotel_id')
data = data.drop(columns = 'avatar_name')
data.head()

,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile,order_requests
0,Yin Yang,Royal Lotus,copenhagen,0,0,0,208,68,30,french,0,1
1,Independant,Independant,copenhagen,0,0,0,85,41,30,french,0,1
2,Yin Yang,8 Premium,copenhagen,1,0,0,86,24,30,french,0,1
3,Morriott International,Morriot,copenhagen,1,1,0,210,73,30,french,0,1
4,Chillton Worldwide,Quadrupletree,copenhagen,0,0,0,215,74,30,french,0,1


In [4]:
data

,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile,order_requests
0,Yin Yang,Royal Lotus,copenhagen,0,0,0,208,68,30,french,0,1
1,Independant,Independant,copenhagen,0,0,0,85,41,30,french,0,1
2,Yin Yang,8 Premium,copenhagen,1,0,0,86,24,30,french,0,1
3,Morriott International,Morriot,copenhagen,1,1,0,210,73,30,french,0,1
4,Chillton Worldwide,Quadrupletree,copenhagen,0,0,0,215,74,30,french,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
353789,Accar Hotels,Safitel,vilnius,0,1,0,187,216,40,austrian,0,1
353790,Morriott International,Corlton,vilnius,0,1,0,181,213,40,austrian,0,1
353791,Morriott International,Corlton,vilnius,1,0,2,185,224,40,austrian,0,1
353792,Independant,Independant,vilnius,0,0,0,60,88,40,austrian,0,1


## on encode les variables par le prix moyen suivant les critères

In [6]:
data = data.drop(['order_requests'],axis = 1)

In [7]:
dict_city = data.groupby("city")["price"].mean().to_dict()
dict_language = data.groupby("language")["price"].mean().to_dict()
dict_brand = data.groupby("brand")["price"].mean().to_dict()
dict_group = data.groupby("group")["price"].mean().to_dict()

In [12]:
from category_encoders import TargetEncoder

In [13]:
encoder = TargetEncoder()
data['city'] = encoder.fit_transform(data['city'],data['price'])
data['language'] = encoder.fit_transform(data['language'],data['price'])
data['brand'] = encoder.fit_transform(data['brand'],data['price'])
data['group'] = encoder.fit_transform(data['group'],data['price'])

/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:129: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


In [14]:
data

,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile
0,151.570010,199.994048,189.984062,0,0,0,208,68,30,159.201699,0
1,99.606709,99.606709,189.984062,0,0,0,85,41,30,159.201699,0
2,151.570010,76.937321,189.984062,1,0,0,86,24,30,159.201699,0
3,226.159936,199.207698,189.984062,1,1,0,210,73,30,159.201699,0
4,216.845527,201.896485,189.984062,0,0,0,215,74,30,159.201699,0
...,...,...,...,...,...,...,...,...,...,...,...
353789,187.713130,288.854029,111.571942,0,1,0,187,216,40,160.929000,0
353790,226.159936,284.659950,111.571942,0,1,0,181,213,40,160.929000,0
353791,226.159936,284.659950,111.571942,1,0,2,185,224,40,160.929000,0
353792,99.606709,99.606709,111.571942,0,0,0,60,88,40,160.929000,0


In [15]:
# On définit la fonction MAPE : Mean Absolute Percentage Error
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [16]:
#Séparation du jeu de données en jeu d'apprentissage et de test
price = data["price"]
price = np.log(price)
X_train, X_test, price_train, price_test = train_test_split(data,price,test_size=0.25,random_state=11)
X_train = X_train.drop(['price'],axis=1)
X_test = X_test.drop(['price'],axis=1)
X_train.shape, price_train.shape

((265345, 10), (265345,))

In [17]:
X_train

,group,brand,city,parking,pool,children_policy,stock,date,language,mobile
208445,99.606709,99.606709,143.068585,0,0,0,8,10,154.084402,1
298501,151.570010,160.779154,170.788382,0,0,0,26,22,154.084402,1
279281,151.570010,160.779154,152.391669,0,1,0,20,17,169.105674,1
319324,226.159936,284.659950,194.308193,0,1,0,1,1,163.209054,1
268342,99.606709,99.606709,92.492271,0,0,0,8,10,163.838595,0
...,...,...,...,...,...,...,...,...,...,...
345357,136.642998,153.235411,146.528060,1,0,0,0,0,143.313247,1
102327,216.845527,156.199740,170.788382,0,0,0,51,30,159.323365,0
203867,226.159936,284.659950,170.788382,1,1,0,14,10,160.450572,0
283728,151.570010,160.779154,160.026393,1,0,0,0,5,159.323365,1


## Random Forest

In [19]:
rf= RandomForestRegressor(n_estimators=400,max_features=2)
rfFit=rf.fit(X_train, price_train)

In [20]:
predictions = rf.predict(X_test)

In [21]:
print("MSE=",mean_squared_error(price_test,predictions))
print("R2=",r2_score(price_test,predictions))
print("MAPE=", MAPE(price_test,predictions))

MSE= 0.009776251461896434
R2= 0.955457619515114
MAPE= 1.0755782569328747


## Interface Gradio

In [137]:
def predict_price(city, language, date, brand, group, parking, pool, mobile, children_policy):

  hotel_features = pd.DataFrame(columns=X_test.columns)

  #en réalité l'utilisateur n'a pas la possibilité de choisir le stock de chambre dns l'hotel, on le fixe donc à 20 ici.
  stock = 20 

  # 1 = l'hotel à 1 parking, 0 sinon
  if parking: 
    park = 1
  else:
    park = 0

  # 1 = l'hotel a une piscine, 0 sinon
  if pool:
    pool = 1
  else:
    pool = 0

  # 1 si la recherche a été faite sur le tel 0 si sur l'ordi
  if mobile:
    mobile = 1
  else:
    mobile = 0

  # le client souhaite un hôtel sans enfant ou non.
  if children_policy == "no_policy":
    children_policy = 0
  elif children_policy == "no_less_12":
    children_policy = 1
  else:
    children_policy = 2

  # on remplit le tableau avec les encoding définis précédemment
  ville = dict_city[city]
  langue = dict_language[language]
  brands = dict_brand[brand]
  groupe = dict_group[group]
  
  hotel_features = hotel_features.append({
                "group" : groupe,
                "brand" : brands,
                "city" : ville, 
                "parking" : park,
                "pool" : pool,
                "children_policy" : children_policy,
                "stock" : stock,
                "date" : date,
                "language" : langue,
                "mobile" : mobile
                }
                , ignore_index=True)
  
  
  estimated_price = np.exp(rf.predict(hotel_features))

  text = "You want to go to " + city + " in " + str(date) + " days. You are " + language + ". You have chosen an hotel" 
  pool = " with a pool and " if pool else " without pool and "
  parking = "with a parking. " if parking else "without parking. "
  brand_group = "You want the brand: " + brand + " and the group: " + group + ". "
  tel = "You want to book on your cellphone. " if mobile else "You want to book on your computer. "
  enfant = "You want a hotel without children." if (children_policy == 1 or children_policy == 2) else "The presence of children does not bother you."
  text = text + pool + parking + brand_group + tel + enfant
  price = "The price is " + str(int(estimated_price))
  print(text)
  return (text,price)

In [138]:
#predict_price(city = "paris", language = "romanian", date = 12, brand = "Ardisson", group = "Independant", parking = 0, pool = 0, mobile = 0, children_policy = 1)

## Gradio v2

In [139]:
import gradio as gr
date = gr.inputs.Slider(minimum=1, maximum=44, step=1)
children_policy = gr.inputs.Radio(choices=["no_policy", "no_less_12", "no_children"])
city1 = gr.inputs.Radio(choices=["amsterdam", "copenhagen", "madrid", "paris", "rome", "sofia", "valletta", "vienna", "vilnius"])
language1 = gr.inputs.Textbox(lines=1, placeholder="Please enter your nationality")
group1 = gr.inputs.Radio(choices = list(dict_group.keys()))
brand1 = gr.inputs.Radio(choices = list(dict_brand.keys()))

demo = gr.Interface(
    fn=predict_price,
    inputs=[city1,language1,date,brand1,group1,"checkbox","checkbox","checkbox",children_policy],
    outputs=["text","text"],
)
demo.launch()

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:183: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please im

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>